## Summary

---

## Imports

In [1]:
import functools
from pathlib import Path

import elaspic2 as el2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.msa_analysis import analyze_msa
from scipy import stats
from sklearn import metrics, model_selection
from tqdm.auto import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-szcdmou8 because the default path (/home/p/pmkim/strokach/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
pd.set_option("max_columns", 1000)
pd.set_option("max_rows", 1000)

## Parameters

In [3]:
NOTEBOOK_DIR = Path("37_humsavar_combine_results").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/37_humsavar_combine_results')

## Load results

In [4]:
DATASET_NAME = "humsavar"
DATASET_PATH = str(
    NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein.parquet")
)
DATASET_ALN_PATH = str(
    NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein-waln.parquet")
)
TASK_COUNT = 612
TASK_COUNT_ALN = 12557

DATASET_NAME, DATASET_PATH, TASK_COUNT, TASK_COUNT_ALN

('humsavar',
 '/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein.parquet',
 612,
 12557)

In [5]:
pfile = pq.ParquetFile(DATASET_PATH)

assert TASK_COUNT == pfile.num_row_groups

In [6]:
total_num_mutations = 0
for row_group in tqdm(range(pfile.num_row_groups)):
    num_mutations = (
        pfile.read_row_group(row_group, columns=["mutation"])
        .to_pandas()["mutation"]
        .str.len()
        .sum()
    )

    total_num_mutations += num_mutations

total_num_mutations

61179

In [7]:
pfile_aln = pq.ParquetFile(DATASET_ALN_PATH)

assert TASK_COUNT_ALN == pfile_aln.num_row_groups

In [8]:
total_num_aln_mutations = 0
for row_group in tqdm(range(pfile_aln.num_row_groups)):
    num_mutations = (
        pfile_aln.read_row_group(row_group, columns=["mutation"])
        .to_pandas()["mutation"]
        .str.len()
        .sum()
    )

    total_num_aln_mutations += num_mutations

total_num_aln_mutations

61174

In [9]:
def get_result_files(result_dir, task_count=TASK_COUNT):
    if "msa_analysis" in str(result_dir):
        prefix = "result"
    else:
        prefix = "shard"

    present_files = []
    missing_files = []
    for i in tqdm(range(1, task_count + 1)):
        path = result_dir.joinpath(f"{prefix}-{i}-of-{task_count}.parquet")
        if path.is_file():
            present_files.append(path)
        else:
            missing_files.append(path)
    return present_files, missing_files

In [10]:
def read_files(files, columns=None):
    dfs = []
    for file in tqdm(files):
        try:
            df = pq.read_table(file, columns=columns).to_pandas(integer_object_nulls=True)
        except pa.ArrowInvalid as error:
            print(error)
            continue
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [11]:
def read_rosetta_files(pfile):
    result_dfs = []
    for task_id in tqdm(range(1, pfile.num_row_groups + 1)):
        row = (
            pfile.read_row_group(task_id - 1, columns=["protein_id", "mutation"])
            .to_pandas()
            .iloc[0]
        )

        input_df = pd.DataFrame(
            {
                "protein_id": [row["protein_id"]] * len(row["mutation"]),
                "mutation": row["mutation"],
            }
        )

        path = NOTEBOOK_DIR.parent.joinpath(
            "31_run_rosetta_ddg",
            DATASET_NAME,
            f"shard-{task_id}-of-{TASK_COUNT}.parquet",
        )
        try:
            rosetta_df = pq.read_table(path).to_pandas()
        except pa.ArrowInvalid:
            print(f"Unreadable file for {path}")
            continue

        if rosetta_df.empty:
            print(f"Empty file for {path}")
            continue

        del rosetta_df["protein_id"]

        result_df = input_df.merge(rosetta_df, on=["mutation"])
        result_dfs.append(result_df)
    return pd.concat(result_dfs, ignore_index=True)

### ProteinSolver

In [12]:
ps_result_dir = NOTEBOOK_DIR.parent.joinpath("31_run_proteinsolver", DATASET_NAME)

In [13]:
present_files, missing_files = get_result_files(ps_result_dir)

assert len(missing_files) == 0
len(present_files), len(missing_files)

(612, 0)

In [14]:
result_ps_df = read_files(present_files)

In [15]:
display(result_ps_df.head(2))
print(len(result_ps_df))

assert len(result_ps_df) == total_num_mutations, len(result_ps_df)

,protein_id,mutation,effect,proteinsolver_core_score_wt,proteinsolver_core_score_mut,proteinsolver_core_features_residue_wt,proteinsolver_core_features_protein_wt,proteinsolver_core_features_residue_mut,proteinsolver_core_features_protein_mut
0,A0A0C5B5G6,K14Q,US,0.070044,0.040160,"[-0.74697345495224, 1.316953420639038, 0.01530...","[-1.1096992492675781, 0.785698652267456, 0.256...","[0.05442849546670914, 0.7043757438659668, -0.2...","[-1.0095055103302002, 0.7263880372047424, 0.24..."
1,A1L188,F55L,US,0.030520,0.088642,"[-2.3987135887145996, 0.10982956737279892, -0....","[-1.777479648590088, -0.03869394585490227, 0.0...","[-4.062615871429443, -0.34137260913848877, -0....","[-1.7952650785446167, -0.047699496150016785, 0..."


61179


### ProtBert

In [16]:
pb_result_dir = NOTEBOOK_DIR.parent.joinpath("31_run_protbert", DATASET_NAME)

In [17]:
present_files, missing_files = get_result_files(pb_result_dir)

assert len(missing_files) == 0
len(present_files), len(missing_files)

(612, 0)

In [18]:
result_pb_df = read_files(present_files)

In [19]:
display(result_pb_df.head(2))
print(len(result_pb_df))

assert len(result_pb_df) == total_num_mutations

,protein_id,mutation,effect,protbert_core_score_wt,protbert_core_score_mut,protbert_core_features_residue_wt,protbert_core_features_protein_wt,protbert_core_features_residue_mut,protbert_core_features_protein_mut
0,A0A0C5B5G6,K14Q,US,0.058932,0.033909,"[-0.005357364658266306, 0.032247915863990784, ...","[0.03903713822364807, -0.016466135159134865, 0...","[-0.0028596159536391497, 0.004915683530271053,...","[0.03650001436471939, -0.025432726368308067, 0..."
1,A1L188,F55L,US,0.093019,0.143134,"[0.1373966485261917, 0.06155325844883919, 0.02...","[0.05456249415874481, 0.024172015488147736, 0....","[0.14880749583244324, 0.08820129185914993, 0.0...","[0.056186407804489136, 0.03005887381732464, 0...."


61179


In [20]:
# proteinsolver_keys = set(result_ps_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# protbert_keys = set(result_pb_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# protbert_missing_keys = proteinsolver_keys - protbert_keys

In [21]:
# missing_task_ids = []
# for row_group in tqdm(range(pfile.num_row_groups)):
#     df = pfile.read_row_group(
#         row_group, columns=["uniprot_id", "mutation"]
#     ).to_pandas()[["uniprot_id", "mutation"]]

#     row_keys = set()
#     for tup in df.itertuples():
#         for mutation in tup.mutation:
#             row_keys.add((tup.uniprot_id, mutation))

#     if protbert_missing_keys & row_keys:
#         missing_task_ids.append(row_group + 1)

# missing_task_ids

### Rosetta

In [22]:
ra_result_dir = NOTEBOOK_DIR.parent.joinpath("31_run_rosetta_ddg", DATASET_NAME)

In [23]:
present_files, missing_files = get_result_files(ra_result_dir)

assert len(missing_files) == 0
len(present_files), len(missing_files)

(612, 0)

In [24]:
result_ra_df = read_files(present_files)

In [25]:
display(result_ra_df.head(2))
print(len(result_ra_df))

assert len(result_ra_df) == total_num_mutations

,protein_id,mutation,effect,rosetta_cart_bonded_wt,rosetta_rama_prepro_wt,rosetta_ref_wt,rosetta_hxl_tors_wt,rosetta_p_aa_pp_wt,rosetta_fa_dun_semi_wt,rosetta_fa_dun_rot_wt,rosetta_fa_dun_dev_wt,rosetta_omega_wt,rosetta_dslf_fa13_wt,rosetta_hbond_sc_wt,rosetta_hbond_bb_sc_wt,rosetta_hbond_lr_bb_wt,rosetta_hbond_sr_bb_wt,rosetta_fa_intra_elec_wt,rosetta_fa_elec_wt,rosetta_lk_ball_bridge_uncpl_wt,rosetta_lk_ball_bridge_wt,rosetta_lk_ball_iso_wt,rosetta_lk_ball_wt,rosetta_fa_intra_sol_xover4_wt,rosetta_fa_intra_rep_xover4_wt,rosetta_fa_intra_atr_xover4_wt,rosetta_fa_sol_wt,rosetta_fa_rep_wt,rosetta_fa_atr_wt,rosetta_dg_wt,rosetta_cart_bonded_change,rosetta_rama_prepro_change,rosetta_ref_change,rosetta_hxl_tors_change,rosetta_p_aa_pp_change,rosetta_fa_dun_semi_change,rosetta_fa_dun_rot_change,rosetta_fa_dun_dev_change,rosetta_omega_change,rosetta_dslf_fa13_change,rosetta_hbond_sc_change,rosetta_hbond_bb_sc_change,rosetta_hbond_lr_bb_change,rosetta_hbond_sr_bb_change,rosetta_fa_intra_elec_change,rosetta_fa_elec_change,rosetta_lk_ball_bridge_uncpl_change,rosetta_lk_ball_bridge_change,rosetta_lk_ball_iso_change,rosetta_lk_ball_change,rosetta_fa_intra_sol_xover4_change,rosetta_fa_intra_rep_xover4_change,rosetta_fa_intra_atr_xover4_change,rosetta_fa_sol_change,rosetta_fa_rep_change,rosetta_fa_atr_change,rosetta_dg_change
0,O97980,H16Y,LB/B,71.898,41.522333,-23.888,21.799667,2.173333,53.990667,33.745333,43.407667,103.566667,0.0,-1.52,0.0,0.0,-5.942,-8.984333,-15.621333,-0.212,-0.040333,-45.462,20.883667,11.069667,3.200,-14.431333,91.535333,4.537333,-99.084667,288.143333,0.427,-0.115333,0.83,0.029333,0.168667,0.097333,0.160667,0.111333,-0.000667,0.0,0.0,0.000,0.0,0.0,-0.063667,-0.588667,-0.174,-0.020667,0.016,0.512333,-0.241667,0.011,-0.411667,-1.572333,0.332667,-1.902333,-2.392333
1,A0A0C5B5G6,K14Q,US,28.964,23.071000,-2.435,0.003000,2.241000,17.628000,15.728000,14.727000,26.549000,0.0,0.00,0.0,0.0,-1.824,-1.160000,-1.069000,-0.295,-0.028000,-16.627,7.977000,4.731000,1.432,-8.683000,32.060000,1.931000,-42.063000,102.859000,1.250,-0.065000,0.61,0.001000,0.002000,2.678000,-1.646000,0.083000,0.149000,0.0,0.0,-1.137,0.0,0.0,-0.253000,-1.836000,0.039,0.001000,-0.808,0.576000,0.172000,0.066,-0.205000,1.423000,0.070000,-1.354000,-0.186000


61179


In [26]:
# proteinsolver_keys = set(result_ps_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# rosetta_keys = set(result_ra_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# rosetta_missing_keys = proteinsolver_keys - rosetta_keys

# assert not rosetta_missing_keys

### MSA

In [27]:
msa_result_dir = NOTEBOOK_DIR.parent.joinpath("31_run_msa_analysis", DATASET_NAME)

In [28]:
present_files, missing_files = get_result_files(msa_result_dir, TASK_COUNT_ALN)

assert len(missing_files) == 0
len(present_files), len(missing_files)

(12557, 0)

In [29]:
result_msa_df = read_files(present_files)

In [30]:
display(result_msa_df.head(2))
print(len(result_msa_df))

assert len(result_msa_df) == total_num_aln_mutations

,protein_id,mutation,effect,msa_count_wt,msa_count_mut,msa_count_total,msa_proba_wt,msa_proba_mut,msa_proba_total,msa_length,msa_proba,msa_H,msa_KL
0,A0A0C5B5G6,K14Q,US,3.0,0.0,3.0,-1.749200,-3.135494,-61.323590,3,-1.749200,-0.000000,2.849038
1,P0CJ72,T13I,LB/B,5.0,9.0,16.0,-1.791759,-1.280934,-66.477422,16,-1.160392,0.862912,1.853562


61174


In [31]:
# proteinsolver_keys = set(result_ps_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# msa_keys = set(result_pb_df[["protein_id", "mutation"]].apply(tuple, axis=1))
# msa_missing_keys = proteinsolver_keys - msa_keys

In [32]:
# missing_task_ids = []
# for row_group in tqdm(range(pfile_aln.num_row_groups)):
#     df = pfile_aln.read_row_group(row_group, columns=["protein_id", "mutation"]).to_pandas()[
#         ["protein_id", "mutation"]
#     ]

#     row_keys = set()
#     for tup in df.itertuples():
#         for mutation in tup.mutation:
#             row_keys.add((tup.protein_id, mutation))

#     if protbert_missing_keys & row_keys:
#         missing_task_ids.append(row_group + 1)

# missing_task_ids

### AlphaFold WT

In [33]:
afwt_result_dir = NOTEBOOK_DIR.parent.joinpath("31_run_alphafold_wt", DATASET_NAME)

In [34]:
present_files, missing_files = get_result_files(afwt_result_dir, TASK_COUNT_ALN)

len(present_files), len(missing_files)

(12363, 194)

In [35]:
protein_mutations_df = pq.read_table(DATASET_ALN_PATH, columns=["protein_id", "mutation", "effect"]).to_pandas()

In [36]:
columns = [
    "protein_id",
    "experimentally_resolved",
    "predicted_lddt",
    "msa_first_row",
    "single",
    "structure_module",
    "plddt",
    "max_predicted_aligned_error",
    "ptm",
]
result_afwt_df = read_files(present_files, columns=columns).set_index("protein_id")

Could not open Parquet input source '/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-10366-of-12557.parquet': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
Could not open Parquet input source '/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-10408-of-12557.parquet': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
Could not open Parquet input source '/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-10452-of-12557.parquet': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
Could not open Parquet input source '/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-10486-of-12557.parquet': Parquet magic bytes not found

In [37]:
display(result_afwt_df.head(2))
print(len(result_afwt_df))

,experimentally_resolved,predicted_lddt,msa_first_row,single,structure_module,plddt,max_predicted_aligned_error,ptm
protein_id,,,,,,,,
A0A0C5B5G6,"[[0.23085688054561615, 0.2812912166118622, 0.4...","[[-4.428761959075928, -4.958076477050781, -4.2...","[[-9.192195892333984, -7.8160529136657715, -3....","[[1.0471205711364746, 50.2182731628418, 9.5015...","[[0.006925947964191437, 0.015513338148593903, ...","[57.36422541091451, 58.55118690727977, 63.5633...",31.75,0.027069
P0CJ72,"[[0.44773733615875244, 0.44976410269737244, 0....","[[-4.362471580505371, -4.686018943786621, -3.9...","[[-5.28439474105835, -4.713302135467529, -8.87...","[[-20.71187400817871, 50.35539245605469, 9.119...","[[0.005178235471248627, 0.02119813859462738, -...","[50.6758737960481, 53.870554884328136, 57.2565...",31.75,0.114342


12322


In [38]:
def get_mutation_embeddings(idx, predictions):
    assert idx >= 0

    def as_residue(x):
        return x[idx].astype(np.float32)

    def as_protein(x):
        return x.mean(axis=0).astype(np.float32)

    embeddings = {
        "experimentally_resolved": predictions["experimentally_resolved"],
        "predicted_lddt": predictions["predicted_lddt"],
        "msa_first_row": predictions["msa_first_row"],
        "single": predictions["single"],
        "structure_module": predictions["structure_module"],
    }

    output = {
        "scores_residue_plddt": predictions["plddt"][idx],
        "scores_protein_plddt": np.mean(predictions["plddt"]),
        "scores_protein_max_predicted_aligned_error": predictions["max_predicted_aligned_error"],
        "scores_proten_ptm": predictions["ptm"],
        **{f"features_residue_{key}": as_residue(value) for key, value in embeddings.items()},
        **{f"features_protein_{key}": as_protein(value) for key, value in embeddings.items()},
    }

    return output


# get_mutation_embeddings(0, result_afwt_df.loc["Q8N5M1"])

In [39]:
protein_mutation_lookup = protein_mutations_df.set_index("protein_id")["mutation"].to_dict()

In [40]:
results = []
for tup in tqdm(result_afwt_df.itertuples(), total=len(result_afwt_df)):
    mutations = protein_mutation_lookup[tup.Index]
    for mutation in mutations:
        features = {
            f"alphafold_core_{key}_wt": value
            for key, value in get_mutation_embeddings(
                int(mutation[1:-1]) - 1, tup._asdict()
            ).items()
        }
        results.append(
            {
                "protein_id": tup.Index,
                "mutation": mutation,
            }
            | features
        )

result_af_df = pd.DataFrame(results)
len(result_af_df)

58306

## Combine results

In [41]:
len(result_ps_df) == len(result_ps_df[["protein_id", "mutation"]].drop_duplicates())

True

In [42]:
result_df = (
    result_ps_df.merge(
        result_pb_df, on=["protein_id", "mutation", "effect"], how="left"
    )
    .merge(result_msa_df, on=["protein_id", "mutation", "effect"], how="left")
    .merge(result_ra_df, on=["protein_id", "mutation", "effect"], how="left")
    .merge(result_af_df, on=["protein_id", "mutation"], how="left")
)

assert len(result_df) == len(result_df[["protein_id", "mutation"]].drop_duplicates())
assert not (
    set(result_ps_df[["protein_id", "mutation"]].apply(tuple, axis=1))
    | set(result_ra_df[["protein_id", "mutation"]].apply(tuple, axis=1))
    | set(result_af_df[["protein_id", "mutation"]].apply(tuple, axis=1))
) - set(result_df[["protein_id", "mutation"]].apply(tuple, axis=1))

In [43]:
display(result_df.head(2))
print(len(result_df))

,protein_id,mutation,effect,proteinsolver_core_score_wt,proteinsolver_core_score_mut,proteinsolver_core_features_residue_wt,proteinsolver_core_features_protein_wt,proteinsolver_core_features_residue_mut,proteinsolver_core_features_protein_mut,protbert_core_score_wt,protbert_core_score_mut,protbert_core_features_residue_wt,protbert_core_features_protein_wt,protbert_core_features_residue_mut,protbert_core_features_protein_mut,msa_count_wt,msa_count_mut,msa_count_total,msa_proba_wt,msa_proba_mut,msa_proba_total,msa_length,msa_proba,msa_H,msa_KL,rosetta_cart_bonded_wt,rosetta_rama_prepro_wt,rosetta_ref_wt,rosetta_hxl_tors_wt,rosetta_p_aa_pp_wt,rosetta_fa_dun_semi_wt,rosetta_fa_dun_rot_wt,rosetta_fa_dun_dev_wt,rosetta_omega_wt,rosetta_dslf_fa13_wt,rosetta_hbond_sc_wt,rosetta_hbond_bb_sc_wt,rosetta_hbond_lr_bb_wt,rosetta_hbond_sr_bb_wt,rosetta_fa_intra_elec_wt,rosetta_fa_elec_wt,rosetta_lk_ball_bridge_uncpl_wt,rosetta_lk_ball_bridge_wt,rosetta_lk_ball_iso_wt,rosetta_lk_ball_wt,rosetta_fa_intra_sol_xover4_wt,rosetta_fa_intra_rep_xover4_wt,rosetta_fa_intra_atr_xover4_wt,rosetta_fa_sol_wt,rosetta_fa_rep_wt,rosetta_fa_atr_wt,rosetta_dg_wt,rosetta_cart_bonded_change,rosetta_rama_prepro_change,rosetta_ref_change,rosetta_hxl_tors_change,rosetta_p_aa_pp_change,rosetta_fa_dun_semi_change,rosetta_fa_dun_rot_change,rosetta_fa_dun_dev_change,rosetta_omega_change,rosetta_dslf_fa13_change,rosetta_hbond_sc_change,rosetta_hbond_bb_sc_change,rosetta_hbond_lr_bb_change,rosetta_hbond_sr_bb_change,rosetta_fa_intra_elec_change,rosetta_fa_elec_change,rosetta_lk_ball_bridge_uncpl_change,rosetta_lk_ball_bridge_change,rosetta_lk_ball_iso_change,rosetta_lk_ball_change,rosetta_fa_intra_sol_xover4_change,rosetta_fa_intra_rep_xover4_change,rosetta_fa_intra_atr_xover4_change,rosetta_fa_sol_change,rosetta_fa_rep_change,rosetta_fa_atr_change,rosetta_dg_change,alphafold_core_scores_residue_plddt_wt,alphafold_core_scores_protein_plddt_wt,alphafold_core_scores_protein_max_predicted_aligned_error_wt,alphafold_core_scores_proten_ptm_wt,alphafold_core_features_residue_experimentally_resolved_wt,alphafold_core_features_residue_predicted_lddt_wt,alphafold_core_features_residue_msa_first_row_wt,alphafold_core_features_residue_single_wt,alphafold_core_features_residue_structure_module_wt,alphafold_core_features_protein_experimentally_resolved_wt,alphafold_core_features_protein_predicted_lddt_wt,alphafold_core_features_protein_msa_first_row_wt,alphafold_core_features_protein_single_wt,alphafold_core_features_protein_structure_module_wt
0,A0A0C5B5G6,K14Q,US,0.070044,0.040160,"[-0.74697345495224, 1.316953420639038, 0.01530...","[-1.1096992492675781, 0.785698652267456, 0.256...","[0.05442849546670914, 0.7043757438659668, -0.2...","[-1.0095055103302002, 0.7263880372047424, 0.24...",0.058932,0.033909,"[-0.005357364658266306, 0.032247915863990784, ...","[0.03903713822364807, -0.016466135159134865, 0...","[-0.0028596159536391497, 0.004915683530271053,...","[0.03650001436471939, -0.025432726368308067, 0...",3.0,0.0,3.0,-1.749200,-3.135494,-61.323590,3.0,-1.749200,-0.000000,2.849038,28.964000,23.071000,-2.435,0.003000,2.241000,17.628000,15.728000,14.727,26.549000,0.000000,0.000000,0.000,0.0,-1.824,-1.160000,-1.069000,-0.295000,-0.028000,-16.627000,7.977000,4.731000,1.432,-8.683,32.060000,1.931,-42.063000,102.859000,1.250000,-0.065000,0.610,0.001000,0.002000,2.678,-1.646000,0.083000,0.149000,0.000000,0.000000,-1.137,0.0,0.000000,-0.253000,-1.836000,0.039,0.001000,-0.808000,0.576,0.172,0.066,-0.205000,1.423000,0.070000,-1.354000,-0.186000,56.406708,62.003188,31.75,0.027069,"[0.0023722572, 0.018356942, 0.10303657, 0.1508...","[-5.2305098, -5.6203985, -4.857919, -3.7483947...","[-3.567015, -5.2736416, -9.009683, -3.05795, 5...","[24.392054, 3.2788322, 21.41848, 21.365389, 6....","[0.005355336, 0.010748506, -0.0057989154, 0.00...","[0.13932697, 0.16886736, 0.2806713, 0.26485315...","[-5.4999924, -6.2546177, -5.384321, -4.418623,...","[-1.6651598, -2.4425163, -5.507199, -0.8483903...","[16

61179


## Calculate EL2 score

In [44]:
model = el2.ELASPIC2()

/opt/conda/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
Some weights of the model checkpoint at /gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2/src/elaspic2/plugins/protbert/data/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expe

In [45]:
proteinsolver_columnms = [c for c in result_df if c.startswith("proteinsolver_core_score")]
protbert_columns = [c for c in result_df if c.startswith("protbert_core_")]

el2_missing = result_df[proteinsolver_columnms + protbert_columns].isnull().any(axis=1)
result_df["el2_score"] = np.nan
result_df.loc[~el2_missing, "el2_score"] = model.predict_mutation_effect(
    [
        t._asdict()
        for t in result_df.loc[~el2_missing, proteinsolver_columnms + protbert_columns].itertuples(
            index=False
        )
    ]
)

## Calculate deltas

In [46]:
for column in list(result_df):
    if not column.endswith("_mut"):
        continue

    column_wt = column.removesuffix("_mut") + "_wt"
    if column_wt not in result_df:
        print(column_wt)
        continue

    column_change = column.removesuffix("_mut") + "_change"
    result_df[column_change] = result_df[column_wt] - result_df[column]
    del result_df[column]

## Encode mutation

In [47]:
# For historic reasons, have to use analyze_msa.RESIDUE_ENCODING_WT/MUT
result_df["aa_wt_onehot"] = (
    result_df["mutation"].str[0].map(analyze_msa.RESIDUE_ENCODING_WT).apply(list)
)
result_df["aa_mut_onehot"] = (
    result_df["mutation"].str[-1].map(analyze_msa.RESIDUE_ENCODING_MUT).apply(list)
)

## Save results

In [48]:
output_file = NOTEBOOK_DIR.joinpath("combined-results.parquet")

output_file

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/workspace/elaspic2-cagi6/notebooks/37_humsavar_combine_results/combined-results.parquet')

In [49]:
pq.write_table(
    pa.Table.from_pandas(result_df, preserve_index=False),
    output_file,
    row_group_size=10_000,
)

## Exploratory data analysis

In [50]:
proteinsolver_columnms = [c for c in result_df if c.startswith("proteinsolver_")]
protbert_columns = [c for c in result_df if c.startswith("protbert_")]
rosetta_columns = [c for c in result_df if c.startswith("rosetta_")]
alphafold_columns = [c for c in result_df if c.startswith("alphafold_")]

In [51]:
result_df["effect"].value_counts()

LB/B    33125
LP/P    21789
US       6265
Name: effect, dtype: int64

In [52]:
effect_map = {
    "LB/B": -1,
    "US": 0,
    "LP/P": 1,
}

result_df["effect_score"] = result_df["effect"].map(effect_map)

In [53]:
score_columns = [
    "el2_score",
    "proteinsolver_core_score_change",
    "protbert_core_score_change",
    "msa_KL",
    "rosetta_dg_change",
    "alphafold_core_scores_residue_plddt_wt",
    #         "alphafold_core_scores_protein_plddt_wt",
    #         "alphafold_core_scores_protein_max_predicted_aligned_error_wt",
    #         "alphafold_core_scores_proten_ptm_wt",
]

df = result_df.dropna(subset=score_columns + ["effect_score"])
df = df[df["effect_score"] != 0].reset_index(drop=True)

for col in score_columns:
    corr = stats.spearmanr(df["effect_score"], df[col])
    auc = metrics.roc_auc_score(df["effect_score"] > 0, df[col])
    print(col, corr[0], auc)

el2_score 0.5855139122395887 0.8461069574744279
proteinsolver_core_score_change 0.35011997874268963 0.7069617101498527
protbert_core_score_change 0.48206117361547335 0.7849543326449486
msa_KL 0.3737434440882958 0.7209259312003657
rosetta_dg_change 0.3281277184343274 0.6939617207211768
alphafold_core_scores_residue_plddt_wt 0.3162389210343963 0.6869340565575879


In [54]:
score_columns = [
    "el2_score",
    "proteinsolver_core_score_change",
    "protbert_core_score_change",
    "msa_KL",
    "rosetta_dg_change",
    "alphafold_core_scores_residue_plddt_wt",
#     "alphafold_core_scores_residue_plddt_change",
#         "alphafold_core_scores_protein_plddt_wt",
#         "alphafold_core_scores_protein_max_predicted_aligned_error_wt",
#         "alphafold_core_scores_proten_ptm_wt",
]

for column in score_columns:
    print(f"{column} {result_df[column].isnull().sum()}")

el2_score 0
proteinsolver_core_score_change 0
protbert_core_score_change 0
msa_KL 5
rosetta_dg_change 0
alphafold_core_scores_residue_plddt_wt 2873
